In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
try:
    import pandas_datareader as web
    import cvxopt as opt
    from cvxopt import solvers    
    from scipy import optimize as op


except:
    !pip install pandas_datareader 
    !pip install cvxopt
    import pandas_datareader as web
    ! pip install scipy
    from scipy import optimize as op


In [2]:
ticker = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
start_date = '2010-06-29'
end_date = '2015-06-30'

In [8]:
data = web.get_data_yahoo(ticker, start_date, end_date)['Adj Close']
data_cap = web.get_quote_yahoo(ticker)['marketCap']

ret = data.pct_change().dropna()

In [10]:
wgt_mkt = data_cap/data_cap.sum()
cov_mkt = ret.cov()

rf = 0.019


AAPL     0.311083
GOOGL    0.191591
MSFT     0.247067
AMZN     0.153089
TSLA     0.097170
Name: marketCap, dtype: float64

In [11]:
view = [('AAPL', 'AMZN', 0.05), ('GOOGL', 'MSFT', 0.02)]

In [24]:
TickerToIndex = dict()
for i, n in enumerate(ticker):
        TickerToIndex[n] = i


In [27]:
TickerToIndex['AAPL']

0

In [66]:
def MatrixQP(view, ticker):
    r, c = len(view), len(ticker)
    Q = np.zeros([r])
    for i in range(r):
        Q[i] = view[i][2]

    TickerToIndex = dict()
    for i, n in enumerate(ticker):
        TickerToIndex[n] = i

    P = np.zeros([r,c])

    for i in range(r):
        P[i, TickerToIndex[view[i][0]]] = 1
        P[i, TickerToIndex[view[i][1]]] = -1

    return Q, P
        

def omega(tau, P, cov):
    return (np.dot(np.dot(np.dot(tau, P), cov), P.transpose()))

In [67]:
Q, P = MatrixQP(view, ticker)

In [71]:
omg = omega(0.025, P, cov_mkt)


$$
\max \sum_{i=1}{n}w^T \mu  - \frac12 w^T \Sigma w \\
s.t. \ \sum{i=1}{n}w_i = 1

$$